In [1]:
from src import dask_preperation as dp
from dask.distributed import Client, wait
from models import custom_RF as md
from load_dataset import custom_lakedepth as data_prep

In [2]:

dask = dp.Dask(1, 8)
c = Client(dask.cluster)
dask.set_client(c)
c

Client Scheduler: tcp://127.0.0.1:36090 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 1 Cores: 1 Memory: 540.49 GB


In [3]:
cols = ['FID', 'Date']
pred = 'Depth_m'
data = data_prep.LakeDepth(42)
#data = data_prep.Data('data/pts_merged_final.csv', pred, cols, 42)

/att/gpfsfs/briskfs01/ppl/cssprad1/projects/rapids_2/rapids_rf_lake_depth/load_dataset/LakeDepth/pts_merged_final.csv
 - from DATA: reading csv into GPU memory
 - from DATA: done reading csv into GPU memory
 - from DATA: dropped column: FID
 - from DATA: dropped column: Date


In [4]:
cv_train, cv_test, l_train, l_test = data.split(0.20)

In [5]:
hyperparameters = {	   'N_ESTIMATORS' : 2000,
					   'SPLIT_ALGO' : 1,
					   'SPLIT_CRITERION' : 2,
					   'BOOTSTRAP' : True,
					   'BOOTSTRAP_FEATURES' : False,
					   'ROWS_SAMPLE' : 1.0,
					   'MAX_DEPTH' : 16,
					   'MAX_LEAVES' : -1,
					   'MAX_FEATURES' : 'auto',
					   'N_BINS' : 8,
					   'MIN_ROWS_PER_NODE' : 2,
					   'MIN_IMPURITY_DECREASE' : 0.0,
					   'ACCURACY_METRIC' : 'mean_ae', # 'mse' #'r2' # 'median_aw' # 
					   'QUANTILEPT' : False,
					   'SEED' :  42,
					   'VERBOSE' : False
					   }


In [6]:
print(hyperparameters)

{'N_ESTIMATORS': 2000, 'SPLIT_ALGO': 1, 'SPLIT_CRITERION': 2, 'BOOTSTRAP': True, 'BOOTSTRAP_FEATURES': False, 'ROWS_SAMPLE': 1.0, 'MAX_DEPTH': 16, 'MAX_LEAVES': -1, 'MAX_FEATURES': 'auto', 'N_BINS': 8, 'MIN_ROWS_PER_NODE': 2, 'MIN_IMPURITY_DECREASE': 0.0, 'ACCURACY_METRIC': 'mean_ae', 'QUANTILEPT': False, 'SEED': 42, 'VERBOSE': False}


In [7]:
rf = md.DaskCumlRF(hyperparameters)

In [8]:
cv_dt, l_dt = dask.distribute(cv_train, l_train)
cv_dte, l_dte = dask.distribute(cv_test, l_test)

In [9]:
%%time

rf.train(cv_dt, l_dt)

CPU times: user 279 ms, sys: 65.7 ms, total: 344 ms
Wall time: 8.38 s


In [10]:
rf.get_metrics(cv_dte, l_test)

ValueError: Cannot create cython.array from NULL pointer

In [ ]:
rf.feature_importances(cv_dt, l_dt, True)